In [ ]:
import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt


def get_dataset_from_testcase(run):
    return run[0:-2]

def get_dataset_size(dataset):
    if dataset == "osm" or dataset == "books":
        return 800_000_000
    else:
        return 200_000_000

dir = '/home/chesetti/Repos/KVector_Merge/sponge/join_all'
runs = []
for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results', 'run', run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        run['dataset_size'] = get_dataset_size(run['dataset'])
        runs.append(run)
df = pd.json_normalize(runs)
print(df.columns)

Index(['command', 'dataset', 'dataset_size', 'result.checksum',
       'result.duration_ns', 'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.num_output_keys', 'result.outer_disk_fetch',
       'result.outer_disk_fetch_size', 'result.outer_total_bytes_fetched',
       'spec.algo', 'spec.algo_name', 'spec.check_checksum', 'spec.common_key',
       'spec.index.epsilon', 'spec.index.search', 'spec.index.type',
       'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk', 'spec.index.leaf_size_in_pages'],
      dtype='object')


In [13]:
def get_index_type(index):
    if "btree" in index:
        return "BTREE"
    if "pgm" in index:
        return "PGM"
    return "NA"

def get_index_variant(index):
    if index == "sort_join" or index == "hash_join":
        return "NA"
    pos = index.find("_")
    return index[pos+1:-1]

df["threads"] = df["spec.num_threads"]
df["epsilon"] = df["spec.index.epsilon"]
df["duration_sec"] = df["result.duration_ns"] / (1000000000)
df["ratio"] = df["spec.common_key"]
df["thput"] = (df["result.num_output_keys"] / df["ratio"]) / (df["result.duration_ns"] / (1000000000))
df["algo"] = df["spec.algo_name"]
df["join_algo"] = df["spec.algo"]
df["index_type"] = df["spec.algo_name"].map(lambda x: get_index_type(x))
df["index_variant"] = df["spec.algo_name"].map(lambda x: get_index_variant(x))
df["inner_bytes_fetched"] = df["result.inner_total_bytes_fetched"]
display(df["algo"].unique())
display(df["dataset"].unique())
display(df["index_type"].unique())


array(['lsj_sampledflatpgm256', 'inlj_flatpgm4096', 'lsj_btree4096',
       'inlj_sampledflatpgm4096', 'inlj_flatpgm1024', 'hash_join',
       'inlj_pgm4096', 'sort_join', 'inlj_pgm1024', 'inlj_btree1024',
       'lsj_sampledflatpgm1024', 'inlj_btree256',
       'inlj_sampledflatpgm1024', 'lsj_pgm256', 'lsj_btree1024',
       'lsj_btree256', 'inlj_flatpgm256', 'lsj_flatpgm1024',
       'inlj_sampledflatpgm256', 'lsj_sampledflatpgm4096', 'inlj_pgm256',
       'lsj_pgm4096', 'lsj_pgm1024', 'lsj_flatpgm4096', 'lsj_flatpgm256',
       'inlj_btree4096'], dtype=object)

array(['fb', 'books', 'osm', 'uniform_dense', 'lognormal',
       'uniform_sparse', 'normal', 'wiki'], dtype=object)

array(['PGM', 'BTREE', 'NA'], dtype=object)

In [3]:
# SingleThread, HJ vs SJ vs INLJ(BTree256) vs INLJ(PGM256) vs LS(BTree256) vs LS(PGM256)
def plot_dataset_join_duration(dataset):
    rows = duckdb.sql(
        "SELECT ratio, threads, algo, MEDIAN(duration_sec) as d, MEDIAN(thput) as t FROM df " 
        "WHERE" 
        "   (algo='hash_join' OR algo='sort_join' OR algo='inlj_btree256' OR algo='inlj_pgm256' OR "
        "   algo='lsj_btree256' OR algo='lsj_flatpgm256')" 
        "   AND threads=1"
        f"   AND dataset='{dataset}'"
        "GROUP BY dataset, ratio, threads, algo"
    ).df()
    return(rows.pivot(index='ratio', values='d', columns=['algo']))

real_datasets = ['fb', 'wiki', 'osm', 'books']
synth_datasets = ['uniform_dense', 'uniform_sparse', 'normal', 'lognormal']

for dataset in real_datasets:
    display(dataset)
    display(plot_dataset_join_duration(dataset))

for dataset in synth_datasets:
    display(dataset)
    display(plot_dataset_join_duration(dataset))

'fb'

algo,hash_join,inlj_btree256,inlj_pgm256,lsj_btree256,lsj_flatpgm256,sort_join
ratio,,,,,,
1,92.803386,53.067202,39.427941,47.774045,25.976550,15.907270
10,49.446850,6.178218,4.912952,6.495105,4.075511,4.825520
100,37.227181,1.664028,1.084884,1.634213,1.199437,3.364797
1000,14.757411,0.259721,0.352090,0.315716,0.285341,3.176952


'wiki'

algo,hash_join,inlj_btree256,inlj_pgm256,lsj_btree256,lsj_flatpgm256,sort_join
ratio,,,,,,
1,37.334655,23.540610,15.039791,20.846256,11.690900,6.969193
10,22.695847,5.059676,3.505500,4.941911,3.507018,2.795611
100,16.403791,0.875864,1.013105,0.882262,0.615757,1.422261
1000,6.730158,0.222480,0.223985,0.222210,0.216568,1.245280


'osm'

...

algo,hash_join,inlj_btree256,inlj_pgm256,lsj_btree256,lsj_flatpgm256,sort_join
ratio,,,,,,
1,91.047514,52.301702,36.662256,48.577115,26.616213,17.222768
10,49.506231,5.985692,4.701640,6.052254,3.820223,4.800495
100,38.401967,1.207014,1.012684,1.198434,1.123010,3.490063
1000,14.258768,0.252391,0.272613,0.254787,0.322005,2.731898


'normal'

algo,hash_join,inlj_btree256,inlj_pgm256,lsj_btree256,lsj_flatpgm256,sort_join
ratio,,,,,,
1,94.056723,54.062095,29.698941,47.962588,26.738348,16.558630
10,47.265245,6.727578,3.945524,5.999719,3.541300,5.026393
100,37.498558,1.192461,0.938083,1.193659,1.072242,2.894747
1000,14.713410,0.243950,0.312992,0.305125,0.305724,2.695198


'lognormal'

algo,hash_join,inlj_btree256,inlj_pgm256,lsj_btree256,lsj_flatpgm256,sort_join
ratio,,,,,,
1,91.594071,54.251634,33.305641,48.218088,24.873390,17.354602
10,50.659646,6.138937,4.297023,6.553460,4.182600,4.595661
100,39.186771,1.189923,1.006116,1.191112,1.075444,2.879593
1000,15.736142,0.239541,0.258540,0.303619,0.306435,2.703660


In [16]:
# Effect of Epsilon on INLJ (BTree and PGM)
def plot_epsilon_inlj(dataset):
    rows = duckdb.sql(
        "SELECT ratio, algo, join_algo, epsilon, MEDIAN(duration_sec) as d, MEDIAN(thput) as t FROM df " 
        "WHERE" 
        "   join_algo = 'inlj'"
        "   AND threads=1"
        f"   AND dataset='{dataset}'"
        "GROUP BY dataset, ratio, algo, join_algo, epsilon"
    ).df()
    return rows.pivot(index='ratio', values='d', columns=['algo', 'epsilon'])

for dataset in real_datasets:
    display(dataset)
    display(plot_epsilon_inlj(dataset))

for dataset in synth_datasets:
    display(dataset)
    display(plot_epsilon_inlj(dataset))

'fb'

algo,inlj_flatpgm4096,inlj_sampledflatpgm4096,inlj_btree1024,inlj_btree4096,inlj_flatpgm1024,inlj_flatpgm256,inlj_pgm1024,inlj_sampledflatpgm256,inlj_btree256,inlj_sampledflatpgm1024,inlj_pgm256,inlj_pgm4096
epsilon,4096.0,4096.0,1024.0,4096.0,1024.0,256.0,1024.0,256.0,256.0,1024.0,256.0,4096.0
ratio,,,,,,,,,,,,
1,36.860210,37.738168,54.454780,53.186117,36.694180,35.735968,38.800283,39.077775,53.067202,39.386363,39.427941,42.717257
10,5.570609,5.405943,6.049074,5.819146,5.253616,4.782187,5.297466,5.984620,6.178218,5.623032,4.912952,5.850861
100,1.833962,1.536952,1.130245,1.093045,1.593616,1.418238,1.199611,1.619190,1.664028,1.396686,1.084884,1.413768
1000,0.775226,0.645554,0.398953,0.510789,0.485742,0.346613,0.491207,0.377439,0.259721,0.427322,0.352090,0.787943


'wiki'

algo,inlj_sampledflatpgm4096,inlj_btree1024,inlj_sampledflatpgm1024,inlj_btree4096,inlj_flatpgm4096,inlj_pgm256,inlj_btree256,inlj_sampledflatpgm256,inlj_pgm1024,inlj_pgm4096,inlj_flatpgm256,inlj_flatpgm1024
epsilon,4096.0,1024.0,1024.0,4096.0,4096.0,256.0,256.0,256.0,1024.0,4096.0,256.0,1024.0
ratio,,,,,,,,,,,,
1,17.307899,22.996426,16.447396,21.834449,15.443946,15.039791,23.540610,15.234307,15.998415,16.979243,13.750013,14.622424
10,4.318298,5.454602,4.215259,4.802448,3.783417,3.505500,5.059676,4.056870,3.834758,4.059837,3.788520,3.527932
100,1.062199,0.822400,1.003368,0.808017,0.932598,1.013105,0.875864,0.931938,0.840494,0.964979,0.718000,0.812739
1000,0.424065,0.276095,0.316107,0.292301,0.400498,0.223985,0.222480,0.306665,0.293112,0.404468,0.222146,0.288934


'osm'

...

algo,inlj_sampledflatpgm1024,inlj_pgm4096,inlj_pgm256,inlj_flatpgm4096,inlj_flatpgm256,inlj_flatpgm1024,inlj_btree1024,inlj_sampledflatpgm256,inlj_pgm1024,inlj_btree4096,inlj_sampledflatpgm4096,inlj_btree256
epsilon,1024.0,4096.0,256.0,4096.0,256.0,1024.0,1024.0,256.0,1024.0,4096.0,4096.0,256.0
ratio,,,,,,,,,,,,
1,35.154202,37.755019,36.662256,35.219507,32.802448,34.293825,55.245917,32.721013,36.347464,53.443362,36.074363,52.301702
10,4.952161,5.537722,4.701640,4.766491,4.369059,4.565168,6.570872,5.214795,5.260598,6.335806,5.277087,5.985692
100,1.256754,1.740371,1.012684,1.264624,0.969279,1.108373,1.071655,1.236354,1.513845,1.442828,1.731077,1.207014
1000,0.395775,0.547029,0.272613,0.549831,0.322530,0.372511,0.376210,0.359669,0.372605,0.488717,0.564094,0.252391


'normal'

algo,inlj_flatpgm1024,inlj_btree1024,inlj_btree256,inlj_flatpgm256,inlj_btree4096,inlj_pgm4096,inlj_flatpgm4096,inlj_sampledflatpgm1024,inlj_sampledflatpgm256,inlj_sampledflatpgm4096,inlj_pgm1024,inlj_pgm256
epsilon,1024.0,1024.0,256.0,256.0,4096.0,4096.0,4096.0,1024.0,256.0,4096.0,1024.0,256.0
ratio,,,,,,,,,,,,
1,29.423931,56.033252,54.062095,27.696246,53.901132,35.679257,31.833445,34.092265,30.905788,35.685901,32.682569,29.698941
10,4.082723,6.414867,6.727578,3.720996,6.607661,5.019567,5.038547,5.635330,5.585452,6.696616,4.791668,3.945524
100,1.510040,1.462578,1.192461,0.913872,1.394548,1.924672,1.433930,1.748598,1.640410,2.257273,1.569407,0.938083
1000,0.422447,0.424038,0.243950,0.306580,0.538436,0.732859,0.734382,0.382800,0.278286,0.582767,0.360370,0.312992


'lognormal'

algo,inlj_pgm1024,inlj_btree256,inlj_btree1024,inlj_flatpgm256,inlj_sampledflatpgm256,inlj_pgm4096,inlj_pgm256,inlj_sampledflatpgm4096,inlj_flatpgm4096,inlj_flatpgm1024,inlj_sampledflatpgm1024,inlj_btree4096
epsilon,1024.0,256.0,1024.0,256.0,256.0,4096.0,256.0,4096.0,4096.0,1024.0,1024.0,4096.0
ratio,,,,,,,,,,,,
1,32.894151,54.251634,55.068840,27.484243,31.509494,37.297779,33.305641,36.831732,32.922510,29.172672,34.070140,52.642450
10,4.741701,6.138937,6.585124,3.737744,5.766262,6.134477,4.297023,7.009174,5.242064,4.229369,6.068160,5.840396
100,1.235123,1.189923,1.449490,0.945974,1.258313,2.442943,1.006116,2.254006,1.744232,1.184007,1.535502,1.015438
1000,0.373348,0.239541,0.427705,0.310651,0.339300,0.837495,0.258540,0.843014,0.622090,0.405674,0.400146,0.438325


In [ ]:
# Effect of Epsilon on INLJ (BTree and PGM)
def plot_epsilon_lsj(dataset):
    rows = duckdb.sql(
        "SELECT ratio, algo, join_algo, epsilon, MEDIAN(duration_sec) as d, MEDIAN(thput) as t FROM df " 
        "WHERE" 
        "   join_algo = 'lsj'"
        "   AND threads=1"
        f"   AND dataset='{dataset}'"
        "GROUP BY dataset, ratio, algo, join_algo, epsilon"
    ).df()
    return rows.pivot(index='ratio', values='d', columns=['algo', 'epsilon'])

for dataset in real_datasets:
    display(dataset)
    display(plot_epsilon_lsj(dataset))

for dataset in synth_datasets:
    display(dataset)
    display(plot_epsilon_lsj(dataset))